In [1]:
import os
import shutil
import pydicom
from PIL import Image

def get_aspect_ratio(file_path):
    """Determina la relación de aspecto de un archivo según su formato"""
    ext = file_path.lower().split('.')[-1]
    
    try:
        if ext == 'dcm':
            dcm = pydicom.dcmread(file_path, stop_before_pixels=True)
            rows = getattr(dcm, "Rows", None)
            cols = getattr(dcm, "Columns", None)
            if rows is None or cols is None:
                return 0
            return cols / rows
        elif ext in ('jpg', 'jpeg', 'png'):
            with Image.open(file_path) as img:
                width, height = img.size
                return width / height
        else:
            return 0
    except Exception as e:
        print(f"Error al procesar {file_path}: {e}")
        return 0

def is_panoramic_by_aspect(file_path, min_ratio=1.5):
    """Considera panorámica toda imagen cuya anchura/altura >= min_ratio."""
    ratio = get_aspect_ratio(file_path)
    return ratio >= min_ratio

# Directorio de salida
no_brackets_output_dir = r"C:\Users\Admn\Documents\rx\data\filtered\panoramic"

if not os.path.exists(no_brackets_output_dir):
    os.makedirs(no_brackets_output_dir)

# Diccionario para llevar la cuenta de nombres repetidos
name_counter = {}

# Lista para almacenar todos los archivos panorámicos encontrados
panoramic_files = []

# Encontrar todas las panorámicas
for root, _, files in os.walk(r"C:\Users\Admn\Documents\rx\data\raw"):
    for fn in files:
        ext = fn.lower().split('.')[-1]
        if ext not in ('dcm', 'jpg', 'jpeg', 'png'):
            continue
        full = os.path.join(root, fn)
        
        if is_panoramic_by_aspect(full, min_ratio=1.5):
            panoramic_files.append((full, fn))

print(f"Encontradas {len(panoramic_files)} panorámicas basadas en aspecto.")

# Copiar todas las panorámicas, renombrando en caso de duplicados
copied_count = 0
renamed_count = 0

for file_path, original_name in panoramic_files:
    base_name, ext = os.path.splitext(original_name)
    
    if original_name in name_counter:
        name_counter[original_name] += 1
        new_name = f"{base_name}_{name_counter[original_name]}{ext}"
        renamed_count += 1
    else:
        name_counter[original_name] = 0
        new_name = original_name
    
    dest_file = os.path.join(no_brackets_output_dir, new_name)
    try:
        shutil.copy2(file_path, dest_file)
        copied_count += 1
        
        # Mostrar progreso
        if copied_count % 100 == 0:
            print(f"Progreso: {copied_count}/{len(panoramic_files)} archivos copiados")
    except Exception as e:
        print(f"Error al copiar {file_path} a {dest_file}: {e}")

# Verificar el resultado final
final_files = os.listdir(no_brackets_output_dir)

print("\n===== RESUMEN =====")
print(f"Total panorámicas identificadas: {len(panoramic_files)}")
print(f"Archivos copiados exitosamente: {copied_count}")
print(f"Archivos renombrados por duplicados: {renamed_count}")
print(f"Total archivos en carpeta destino: {len(final_files)}")
print(f"Archivos guardados en: {no_brackets_output_dir}")

Encontradas 3534 panorámicas basadas en aspecto.
Progreso: 100/3534 archivos copiados
Progreso: 200/3534 archivos copiados
Progreso: 300/3534 archivos copiados
Progreso: 400/3534 archivos copiados
Progreso: 500/3534 archivos copiados
Progreso: 600/3534 archivos copiados
Progreso: 700/3534 archivos copiados
Progreso: 800/3534 archivos copiados
Progreso: 900/3534 archivos copiados
Progreso: 1000/3534 archivos copiados
Progreso: 1100/3534 archivos copiados
Progreso: 1200/3534 archivos copiados
Progreso: 1300/3534 archivos copiados
Progreso: 1400/3534 archivos copiados
Progreso: 1500/3534 archivos copiados
Progreso: 1600/3534 archivos copiados
Progreso: 1700/3534 archivos copiados
Progreso: 1800/3534 archivos copiados
Progreso: 1900/3534 archivos copiados
Progreso: 2000/3534 archivos copiados
Progreso: 2100/3534 archivos copiados
Progreso: 2200/3534 archivos copiados
Progreso: 2300/3534 archivos copiados
Progreso: 2400/3534 archivos copiados
Progreso: 2500/3534 archivos copiados
Progreso: 

In [3]:
import os
import torch
import pydicom
import numpy as np
from PIL import Image
from torchvision import transforms
from torch import nn
from torchvision.models import resnet18
from torch.utils.data import Dataset, DataLoader
import random


class SiameseNet(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.backbone = resnet18(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, embedding_dim)
    
    def forward_once(self, x):
        return self.backbone(x)
    
    def forward(self, x1, x2):
        # Para entrenamiento, procesamos pares de imágenes
        emb1 = self.forward_once(x1)
        emb2 = self.forward_once(x2)
        return emb1, emb2
    
    def embedding(self, x):
        return self.forward_once(x)

# --- Función para cargar imagen ---
def load_image(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == '.dcm':
        dcm = pydicom.dcmread(path)
        arr = dcm.pixel_array.astype(np.float32)
        # Normalizamos a 0-255
        arr = (arr - arr.min()) * (255.0 / (arr.max() - arr.min()))
        arr = np.clip(arr, 0, 255).astype(np.uint8)
        img = Image.fromarray(arr)
    else:
        # Carga con PIL para JPG, PNG, etc.
        img = Image.open(path).convert('L')
    return img

transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2,contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Dataset para entrenamiento de red siamesa ---
class SiameseDataset(Dataset):
    def __init__(self, leche_dir, def_dir, transform=None):
        self.leche_files = [os.path.join(leche_dir, f) for f in os.listdir(leche_dir)
                          if f.lower().endswith(('.jpg', '.jpeg', '.png', '.dcm'))]
        self.def_files = [os.path.join(def_dir, f) for f in os.listdir(def_dir)
                        if f.lower().endswith(('.jpg', '.jpeg', '.png', '.dcm'))]
        self.transform = transform
        
    def __len__(self):
        # Generamos pares: mismo tipo (positivos) y diferente tipo (negativos)
        return len(self.leche_files) + len(self.def_files)
    
    def __getitem__(self, idx):
        # 50% probabilidad de obtener un par positivo, 50% negativo
        is_same = random.random() > 0.5
        
        if idx < len(self.leche_files):
            img1_path = self.leche_files[idx]
            if is_same:
                # Par positivo: ambos son dientes de leche
                img2_path = random.choice(self.leche_files)
                label = 1  # mismo tipo = 1
            else:
                # Par negativo: uno leche, otro definitivo
                img2_path = random.choice(self.def_files)
                label = 0  # diferente tipo = 0
        else:
            img1_path = self.def_files[idx - len(self.leche_files)]
            if is_same:
                # Par positivo: ambos son dientes definitivos
                img2_path = random.choice(self.def_files)
                label = 1  # mismo tipo = 1
            else:
                # Par negativo: uno definitivo, otro leche
                img2_path = random.choice(self.leche_files)
                label = 0  # diferente tipo = 0
        
        img1 = load_image(img1_path)
        img2 = load_image(img2_path)
        
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        
        return img1, img2, torch.FloatTensor([label])

# --- Función de pérdida contrastiva ---
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.6): # 2.0
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
    
    def forward(self, output1, output2, label):
        # Calculamos la distancia euclidiana entre las salidas
        euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)
        # Pérdida contrastiva
        loss = torch.mean(label * euclidean_distance.pow(2) + (1 - label) * torch.clamp(self.margin - euclidean_distance, min=0).pow(2))
        return loss

# --- Directorios con ejemplos etiquetados ---
leche_dir = r"C:\Users\Admn\Documents\rx\data\training\milk-teeth"  # Ruta a carpeta con ejemplos de dientes de leche
def_dir = r"C:\Users\Admn\Documents\rx\data\training\permanent-teeth"  # Ruta a carpeta con ejemplos de dientes definitivos

# --- Configuración del entrenamiento ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")
model = SiameseNet(embedding_dim=128).to(device)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
num_epochs = 20
batch_size = 8

# --- Preparar dataset y dataloader ---
dataset = SiameseDataset(leche_dir, def_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# --- Entrenamiento ---
print("Comenzando entrenamiento...")
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (img1, img2, label) in enumerate(dataloader):
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        output1, output2 = model(img1, img2)
        
        # Calcular pérdida
        loss = criterion(output1, output2, label)
        
        # Backward y optimización
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Mostrar estadísticas
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader):.4f}")

# --- Guardar el modelo entrenado ---
torch.save(model.state_dict(), 'siamese_weights.pth')
print("Entrenamiento completado y modelo guardado como 'siamese_weights.pth'")


Usando dispositivo: cuda
Comenzando entrenamiento...
Epoch 1/20, Loss: 2.7468
Epoch 2/20, Loss: 0.6857
Epoch 3/20, Loss: 0.6802
Epoch 4/20, Loss: 0.6716
Epoch 5/20, Loss: 0.6426
Epoch 6/20, Loss: 0.6541
Epoch 7/20, Loss: 0.6547
Epoch 8/20, Loss: 0.6546
Epoch 9/20, Loss: 0.6687
Epoch 10/20, Loss: 0.6667
Epoch 11/20, Loss: 0.7171
Epoch 12/20, Loss: 0.6772
Epoch 13/20, Loss: 0.7045
Epoch 14/20, Loss: 0.6981
Epoch 15/20, Loss: 0.6648
Epoch 16/20, Loss: 0.6587
Epoch 17/20, Loss: 0.6855
Epoch 18/20, Loss: 0.6669
Epoch 19/20, Loss: 0.6429
Epoch 20/20, Loss: 0.6866
Entrenamiento completado y modelo guardado como 'siamese_weights.pth'


In [4]:
import os
import torch
import pydicom
import numpy as np
import shutil
from PIL import Image
from torchvision import transforms
from torch import nn
from torchvision.models import resnet18

# --- Definición de la red siamés ---
class SiameseNet(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.backbone = resnet18(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, embedding_dim)

    def forward_once(self, x):
        return self.backbone(x)

    def embedding(self, x):
        return self.forward_once(x)

# --- Función para cargar imagen (DICOM o JPG/PNG) como PIL Image ---
def load_image(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == '.dcm':
        dcm = pydicom.dcmread(path)
        arr = dcm.pixel_array.astype(np.float32)
        # Normalizamos a 0-255
        arr = (arr - arr.min()) * (255.0 / (arr.max() - arr.min()))
        arr = np.clip(arr, 0, 255).astype(np.uint8)
        img = Image.fromarray(arr)
    else:
        # Carga con PIL para JPG, PNG, etc.
        img = Image.open(path).convert('L')
    return img

# --- Transformaciones ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def get_class_embedding(directory):
    embeddings = []
    for file in os.listdir(directory):
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.dcm')):
            path = os.path.join(directory, file)
            try:
                img = transform(load_image(path)).unsqueeze(0).to(device)
                with torch.no_grad():
                    emb = model.embedding(img)
                    embeddings.append(emb)
            except Exception as e:
                print(f"Error procesando {file}: {e}")
    
    if not embeddings:
        raise ValueError(f"No se pudieron cargar imágenes de {directory}")
    
    return torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)

# --- Carga del modelo y pesos entrenados ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNet(embedding_dim=128).to(device)
state = torch.load('siamese_weights.pth', map_location=device)
model.load_state_dict(state)
model.eval()

leche_dir = r"C:\Users\Admn\Documents\rx\data\training\milk-teeth"  # Ruta a carpeta con ejemplos de dientes de leche
def_dir = r"C:\Users\Admn\Documents\rx\data\training\permanent-teeth"  # Ruta a carpeta con ejemplos de dientes definitivos

with torch.no_grad():
    proto_leche_emb = get_class_embedding(leche_dir)
    proto_def_emb = get_class_embedding(def_dir)

# --- Función de clasificación general ---
def classify_teeth_type(query_path, threshold=None):
    img = transform(load_image(query_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        q_emb = model.embedding(img)
        d_leche = torch.norm(q_emb - proto_leche_emb, p=2).item()
        d_def = torch.norm(q_emb - proto_def_emb, p=2).item()
    label = 'leche' if d_leche < d_def else 'definitivos'
    return (label, min(d_leche, d_def)) if threshold else label

# --- Definir carpetas de destino ---
milk_teeth_dir = r"C:\Users\Admn\Documents\rx\data\filtered\milk-teeth"
permanent_teeth_dir = r"C:\Users\Admn\Documents\rx\data\filtered\permanent-teeth"

# Crear las carpetas si no existen
if not os.path.exists(milk_teeth_dir):
    os.makedirs(milk_teeth_dir)
if not os.path.exists(permanent_teeth_dir):
    os.makedirs(permanent_teeth_dir)

# --- Iteración sobre carpeta con imágenes mixtas ---
raw_dir = r"C:\Users\Admn\Documents\rx\data\filtered\panoramic"
milk_teeth_files = []
permanent_teeth_files = []

for root, _, files in os.walk(raw_dir):
    print(f"Procesando carpeta: {root}")
    print(f"Total de archivos: {len(files)}")
    for fn in files:
        ext = fn.lower().split('.')[-1]
        if ext not in ('dcm', 'jpg', 'jpeg', 'png'):
            continue
        full = os.path.join(root, fn)
        
        # Clasificar la imagen
        label = classify_teeth_type(full)
        
        # Guardar en la carpeta correspondiente
        if label == 'leche':
            milk_teeth_files.append(full)
            dest_file = os.path.join(milk_teeth_dir, fn)
            shutil.copy2(full, dest_file)
            print(f"Copiado a milk-teeth: {fn}")
        else:  # 'definitivos'
            permanent_teeth_files.append(full)
            dest_file = os.path.join(permanent_teeth_dir, fn)
            shutil.copy2(full, dest_file)
            print(f"Copiado a permanent-teeth: {fn}")

print(f"Total panorámicas con dientes de leche: {len(milk_teeth_files)}")
print(f"Total panorámicas con dientes permanentes: {len(permanent_teeth_files)}")
print(f"Archivos de dientes de leche guardados en: {milk_teeth_dir}")
print(f"Archivos de dientes permanentes guardados en: {permanent_teeth_dir}")

Procesando carpeta: C:\Users\Admn\Documents\rx\data\filtered\panoramic
Total de archivos: 3526
Copiado a milk-teeth: 02 REYES BUSTOS PASCALE 09-10-10.jpg
Copiado a permanent-teeth: ABARZUA MONTECINOS MAURICIO 18-02-02.jpg
Copiado a milk-teeth: ABARZUA VIDAL MARTIN (2).jpg
Copiado a permanent-teeth: ABARZUA VIDAL MARTIN.jpg
Copiado a permanent-teeth: ABARZUA VIDAL MARTIN_1.jpg
Copiado a permanent-teeth: ABARZUA VIDAL MARTIN_2.jpg
Copiado a permanent-teeth: ABARZUA VIDAL SEBASTIAN 14-06-06 (2).jpg
Copiado a permanent-teeth: ABELLO BASUALTO ANAHI  (1).jpg
Copiado a permanent-teeth: ABELLO GUZMAN LISSETTE 19-12-99.jpg
Copiado a permanent-teeth: ABURTO ABURTO MAXIMILIANO (2).jpg
Copiado a permanent-teeth: ABURTO ABURTO MAXIMILIANO 08-03-06 (2).jpg
Copiado a permanent-teeth: ABURTO ABURTO MAXIMILIANO.jpg
Copiado a permanent-teeth: ABURTO CIFUENTES FRANCINY 25-01-01.jpg
Copiado a permanent-teeth: ABURTO CIFUENTES FRANCINY.jpg
Copiado a permanent-teeth: ABURTO GALLEGOS CAROLINA.jpg
Copiado a p

In [5]:
import os
import torch
import pydicom
import numpy as np
from PIL import Image
from torchvision import transforms, models
from torch import nn
from torch.utils.data import Dataset, DataLoader
import shutil

# Función para cargar imágenes (misma que ya tienes)
def load_image(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == '.dcm':
        dcm = pydicom.dcmread(path)
        arr = dcm.pixel_array.astype(np.float32)
        arr = (arr - arr.min()) * (255.0 / (arr.max() - arr.min()))
        arr = np.clip(arr, 0, 255).astype(np.uint8)
        img = Image.fromarray(arr)
    else:
        img = Image.open(path).convert('L')
    return img

# Dataset para la clasificación de brackets
class BracketsDataset(Dataset):
    def __init__(self, brackets_dir, no_brackets_dir, transform=None):
        self.brackets_files = [os.path.join(brackets_dir, f) for f in os.listdir(brackets_dir)
                             if f.lower().endswith(('.jpg', '.jpeg', '.png', '.dcm'))]
        self.no_brackets_files = [os.path.join(no_brackets_dir, f) for f in os.listdir(no_brackets_dir)
                                if f.lower().endswith(('.jpg', '.jpeg', '.png', '.dcm'))]
        self.transform = transform
        self.all_files = [(f, 1) for f in self.brackets_files] + [(f, 0) for f in self.no_brackets_files]
        
    def __len__(self):
        return len(self.all_files)
    
    def __getitem__(self, idx):
        img_path, label = self.all_files[idx]
        img = load_image(img_path)
        
        if self.transform:
            img = self.transform(img)
        
        return img, torch.tensor(label, dtype=torch.float32)
    
# Modelo para detectar brackets
class BracketsDetector(nn.Module):
    def __init__(self):
        super().__init__()
        # Usar un modelo pre-entrenado más pequeño para evitar sobreajuste
        self.model = models.resnet18(pretrained=True)
        
        # Congelar la mayoría de las capas para evitar sobreajuste
        for param in list(self.model.parameters())[:-20]:  # Congelar todo excepto las últimas capas
            param.requires_grad = False
            
        # Modificar la capa final para clasificación binaria
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(0.4),  # Añadir dropout para reducir sobreajuste
            nn.Linear(num_ftrs, 1),
            nn.Sigmoid()  # Para salida binaria
        )
    
    def forward(self, x):
        return self.model(x)
# Transformaciones con data augmentation
brackets_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Configurar directorios
brackets_dir = r"C:\Users\Admn\Documents\rx\data\training\brackets"          # Imágenes con brackets
no_brackets_dir = r"C:\Users\Admn\Documents\rx\data\training\no-brackets"    # Imágenes sin brackets

# Preparar dataset y dataloader
dataset = BracketsDataset(brackets_dir, no_brackets_dir, transform=brackets_transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Configuración del entrenamiento
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BracketsDetector().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)
num_epochs = 15

# Entrenamiento
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Evaluar en conjunto de prueba
    model.eval()
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Acc: {100*correct/total:.2f}%, Test Acc: {100*test_correct/test_total:.2f}%")

# Guardar el modelo
torch.save(model.state_dict(), 'brackets_detector.pth')

def filter_milk_teeth_without_brackets(milk_teeth_dir, no_brackets_output_dir, brackets_output_dir):
    # Cargar el modelo de detección de brackets
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    brackets_model = BracketsDetector().to(device)
    brackets_model.load_state_dict(torch.load('brackets_detector.pth', map_location=device))
    brackets_model.eval()
    
    # Transformación para evaluación 
    eval_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Crear directorio de salida si no existe
    if not os.path.exists(no_brackets_output_dir):
        os.makedirs(no_brackets_output_dir)

    if not os.path.exists(brackets_output_dir):
        os.makedirs(brackets_output_dir)
    
    # Contador para estadísticas
    total = 0
    without_brackets = 0
    
    # Procesar cada imagen
    for filename in os.listdir(milk_teeth_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.dcm')):
            filepath = os.path.join(milk_teeth_dir, filename)
            total += 1
            
            # Cargar y preprocesar imagen
            img = load_image(filepath)
            img_tensor = eval_transform(img).unsqueeze(0).to(device)
            
            # Detectar brackets
            with torch.no_grad():
                output = brackets_model(img_tensor)
                has_brackets = output.item() > 0.5
            
            # Si no tiene brackets, copiar a la carpeta de salida
            if not has_brackets:
                without_brackets += 1
                shutil.copy2(filepath, os.path.join(no_brackets_output_dir, filename))
                print(f"Copiado: {filename} (sin brackets)")

            else:
                print(f"Copiado: {filename} (con brackets)")
                shutil.copy2(filepath, os.path.join(brackets_output_dir, filename))
    
    print(f"Procesamiento completado. {without_brackets}/{total} imágenes sin brackets.")


milk_teeth_dir = r"C:\Users\Admn\Documents\rx\data\filtered\milk-teeth"  # Carpeta con todas las imágenes de dientes de leche
no_brackets_output_dir = r"C:\Users\Admn\Documents\rx\data\filtered\milk-teeth-no-brackets"  # Carpeta para guardar imágenes sin brackets
brackets_output_dir = r"C:\Users\Admn\Documents\rx\data\filtered\milk-teeth-with-brackets"  # Carpeta para guardar imágenes con brackets
filter_milk_teeth_without_brackets(milk_teeth_dir, no_brackets_output_dir, brackets_output_dir)

Epoch 1/15, Loss: 0.6651, Train Acc: 55.41%, Test Acc: 50.00%
Epoch 2/15, Loss: 0.4062, Train Acc: 81.53%, Test Acc: 87.50%
Epoch 3/15, Loss: 0.3571, Train Acc: 86.62%, Test Acc: 100.00%
Epoch 4/15, Loss: 0.2589, Train Acc: 88.54%, Test Acc: 97.50%
Epoch 5/15, Loss: 0.2110, Train Acc: 91.72%, Test Acc: 97.50%
Epoch 6/15, Loss: 0.2123, Train Acc: 92.36%, Test Acc: 97.50%
Epoch 7/15, Loss: 0.1350, Train Acc: 95.54%, Test Acc: 95.00%
Epoch 8/15, Loss: 0.2053, Train Acc: 91.72%, Test Acc: 97.50%
Epoch 9/15, Loss: 0.1112, Train Acc: 97.45%, Test Acc: 97.50%
Epoch 10/15, Loss: 0.1471, Train Acc: 92.36%, Test Acc: 95.00%
Epoch 11/15, Loss: 0.2140, Train Acc: 91.08%, Test Acc: 85.00%
Epoch 12/15, Loss: 0.0768, Train Acc: 96.82%, Test Acc: 95.00%
Epoch 13/15, Loss: 0.1142, Train Acc: 94.27%, Test Acc: 97.50%
Epoch 14/15, Loss: 0.0725, Train Acc: 98.73%, Test Acc: 97.50%
Epoch 15/15, Loss: 0.0492, Train Acc: 98.09%, Test Acc: 97.50%
Copiado: 02 REYES BUSTOS PASCALE 09-10-10.jpg (sin brackets)
Co